In [2]:
import json
import grequests
import sys
import requests
import pandas as pd

In [3]:
# Fly base accession Filter 
id_conv = pd.read_csv("/home/meyer/droso-analysis/data/flybase_conversion/fbgn_NAseq_Uniprot_fb_2020_02.tsv", sep="\t", skiprows=4)
id_mismatch = pd.read_csv("/home/meyer/droso-analysis/data/mismatch-analysis/mismatch.id", header=None)
id_tr = pd.read_csv("/home/meyer/droso-analysis/data/flybase_conversion/fbgn_fbtr_fbpp_fb_2020_02.tsv", sep="\t", skiprows=4)
id_merged = id_mismatch.merge(id_conv, how="left", left_on=0, right_on="UniprotKB/Swiss-Prot/TrEMBL_accession")
id_merged = id_merged.iloc[:, [0, 3]]
id_merged.columns = ["From", "To"]
id_merged.dropna(inplace=True)
#id_merged.to_csv("/home/meyer/droso-analysis/data/mismatch-analysis/transcript_fb.tab", sep="\t", index=False)

In [20]:
finaldf = id_merged.merge(id_tr, how="left", left_on="To", right_on="## FlyBase_FBgn")
finaldf.drop(labels="## FlyBase_FBgn", axis=1, inplace=True)
finaldf.columns = ["Uniprot", "Gene Name", "Transcript", "FB prot"]
finaldf

,Uniprot,Gene Name,Transcript,FB prot
0,A0A0Q9WFF6,FBgn0203889,FBtr0441813,FBpp0398324
1,B4L587,FBgn0144410,NaN,NaN
2,Q29MZ1,FBgn0076481,FBtr0368238,FBpp0331042
3,Q29MZ1,FBgn0076481,FBtr0372119,FBpp0334173
4,Q29MZ1,FBgn0076481,FBtr0373223,FBpp0335045
...,...,...,...,...
13184,B4M645,FBgn0197738,FBtr0442511,FBpp0398976
13185,B4M645,FBgn0197738,FBtr0443330,FBpp0399735
13186,B4HER4,FBgn0181308,NaN,NaN
13187,B4N7Y2,FBgn0213149,NaN,NaN


In [80]:
ID_file = pd.read_csv("../../data/mismatch-analysis/transcript_fb.tab", sep="\t")
url = "https://api.flybase.org/api/v1.0/sequence/id/bulk"
headers = {"Content-Type": "application/x-www-form-urlencoded",
            "Accept": "text/plain"}
error = True
params = []
params.append({"ids": ID_file.iloc[:, 1].tolist(), "type": "CDS"})
rs = [grequests.post(url, headers=headers, data=params[0])]
all_response = grequests.map(rs, size=1)
print(all_response[0])
f = open("../../data/mismatch-analysis/CDS.fasta", "w")
f.write(all_response[0].text)
f.close()

<Response [200]>


In [19]:
# IMPORTANT: you need to  meyer@milex:~/droso-analysis/data/raw/uniprot-blast/results$ cat *.fasta > all_sequence.fasta
# in order to be able to do that
import pandas as pd
import sys
import math
try:
    sys.path.append('~/droso-analysis/src/')
    from Generate_Exon_Map_4 import *
except:
    pass

ID_file = pd.read_csv("../../data/mismatch-analysis/transcript_fb.tab", sep="\t")
my_CDS = fasta2List("../../data/mismatch-analysis/CDS.fasta")
Prot_list = fasta2List("/home/meyer/droso-analysis/data/raw/uniprot-blast/results/all_sequence.fasta")

# Dataframe pour CDS
CDS_ensembl_key = []
CDS_uniprot_key = []
CDS_seq = []
for key, val in my_CDS.items():
    myKey = key[1:].split(" ")
    CDS_ensembl_key.append(myKey[-4].split(",")[0][7:])
    CDS_uniprot_key.append(ID_file.loc[ID_file["To"] == myKey[-4].split(",")[0][7:]].iloc[0,0])
    CDS_seq.append(val)
dict = {"CDS_ensembl_key": CDS_ensembl_key,
        "CDS_uniprot_key": CDS_uniprot_key, "CDS_seq": CDS_seq}
df_CDS = pd.DataFrame(dict)

# Dataframe pour Protein
uniprot_ID = []
uniprot_Seq = []
for key, val in Prot_list.items():
    myKey = key[1:].split(" ")
    uniprot_ID.append(myKey[0])
    uniprot_Seq.append(val)
dict = {"uniprot_ID": uniprot_ID, "uniprot_Seq": uniprot_Seq}
df_prot = pd.DataFrame(dict)

In [33]:
len(my_CDS)

8517

In [20]:
df_CDS

,CDS_ensembl_key,CDS_uniprot_key,CDS_seq
0,FBgn0210482,B4LY15,ATGGCCGAGGATACCGAGTACAAAAAGTTACCCGTTGAGGAGCGCT...
1,FBgn0183340,B4QG40,ATGCCACAGTCAGAGGGCGGCTATGTATCGCTGCCGGCCGTCAATG...
2,FBgn0244037,A0A0R3P099,ATGCCGAAGATCTTCCTGATCAAGAACCGCCTGCATCAGCAGCAAC...
3,FBgn0244201,A0A0R3P0J9,ATGGATCGTAAGGCTGAGCTGGAACGAAAAAAGGCCAAATTGGCGG...
4,FBgn0099929,B3MT55,ATGCATTCCTTGGCAATAGTCGGACTTTTGTGGAGTGGCCTGATGG...
...,...,...,...
8512,FBgn0207449,B4LIT0,ATGCATATGTATATATATCCACCACATCGACATCGATACATCTGGA...
8513,FBgn0205778,B4M2M8,ATGTACAAACCTGACGGTATCACCCACGGTGGAGGCAACGGCAGCA...
8514,FBgn0194386,B4Q5L2,ATGAAGTATTCAACAAGTCGACTAAACATTCTGTGCCTGCTGGGAT...
8515,FBgn0203396,B4MDA7,ATGTCGACAGCACAGAGATCGTTTGTACAAAAAGTTTCAAAGCAAC...


In [37]:
# On cherche les correspondance de taille
f = open("../../data/mismatch-analysis/CDS_filt.fasta", "w")
f.write("From\tTo\n")
counter=0
for index, row in ID_file.iloc[:100, :].iterrows():
    try:
        CDS = df_CDS.loc[df_CDS["CDS_ensembl_key"] == row[1]].iloc[0, 2]
        prot = df_prot.loc[df_prot["uniprot_ID"] == row[0]].iloc[0, 1]
    except:
        print(row[1])
        print(row[0])
        continue
    if (math.ceil((len(CDS)-3)/3) == len(prot)) or \
        (math.floor((len(CDS)-3)/3) == len(prot)) or \
        (len(CDS)/3 == len(prot)) or \
            ((len(CDS)-3)/3 == len(prot)):
        #f.write(row[0] + "\t" + row[1]+"\n")
        pass
    else:
        f.write(row[0] + "\t" + row[1]+"\n")

f.close()

217047
A0A0Q9WXJ1
FBgn0157269
B4G810
FBgn0140983
A0A0Q9XDH0
FBgn0164135
B4GSK1
FBgn0173128
B4I2L5
FBgn0164148
B4GSM6
FBgn0226452
B4N0E4
FBgn0164032
B4GSP2
FBgn0140621
B4KLD4
FBgn0134858
B4KF94
FBgn0226710
B4N003
FBgn0173286
B4I2S9
FBgn0173292
B4I2U0
FBgn0156922
B4G8N3
FBgn0173292
B4I2U0
FBgn0157173
B4G6Z3
FBgn0139842
B4KIL4
FBgn0173080
B4I2U8
FBgn0173306
B4I2W4
FBgn0173065
B4I2X1
FBgn0173064
B4I2X3
FBgn0173317
B4I2Y3
FBgn0217452
B4MV76
FBgn0173047
B4I303
FBgn0146622
B4KJY7
FBgn0173339
B4I333
FBgn0217490
B4MVE3
FBgn0140856
B4KGR5
FBgn0156433
B4G8I3
FBgn0139770
B4KHD9
FBgn0138852
B4KFP8
FBgn0226573
B4N0N8
FBgn0226655
B4MZP4
FBgn0153057
B4GPW3
FBgn0140708
B4KES8
FBgn0226654
B4MZP3
FBgn0173375
B4I385
FBgn0152724
B4GPV7
FBgn0225885
B4MTX2
FBgn0153629
B4GQE8
FBgn0140628
B4KLF2
FBgn0172985
B4I3B7
FBgn0153574
B4GQC8
FBgn0156969
B4G8X4
FBgn0156241
B4G9J4
FBgn0140851
B4KGR0
FBgn0226550
A0A0Q9X1N6
FBgn0157092
B4G9J0
FBgn0138197
B4L2V1
FBgn0166671
B4ILD1
FBgn0166668
B4ILD2
FBgn0137898
B4L2V6
FBgn0